In [1]:
# Importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import defaultdict


In [2]:
# Loading data
yearly_train = pd.read_csv('train/Yearly-train.csv')
quarterly_train = pd.read_csv('train/Quarterly-train.csv')
monthly_train = pd.read_csv('train/Monthly-train.csv')
weekly_train = pd.read_csv('train/Weekly-train.csv')
daily_train = pd.read_csv('train/Daily-train.csv')
hourly_train = pd.read_csv('train/Hourly-train.csv')

In [4]:
train_data = [yearly_train, quarterly_train, monthly_train, weekly_train, daily_train, hourly_train]

for data in train_data:
    del data['Unnamed: 0']

In [5]:
# Setting parameters

params = {'Yearly': (30, 2000), 'Quarterly': (60, 2000),
          'Monthly': (156, 2000), 'Weekly': (315, 359) , 'Daily': (500, 2000), 'Hourly': (720, 414)}

data_sample = []

for data in tqdm(train_data):
    param = params[data.label.iloc[0]]
    
#   Choosing params
    length = param[0]
    size = param[1]
    
    # Leave out short series
    data_len = data[data.length >= length]
    
#     If sample is too small, take it all
    if len(data_len) < size:
        size = len(data_len)

        
#   Else take random sample
    choice = np.random.choice(len(data_len), size = size, replace = False)
    data_len = data_len.iloc[choice]
    len_labs = data_len['label']
    len_lens = data_len['length']
    
#     Trimming series
    data_len = data_len.iloc[:, :length]
    data_len.index = np.arange(len(data_len))

    data_sample.append(data_len.transpose())
    


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.12it/s]


In [6]:
for i in range(6):
#   Standartization of series
    data_sample[i] = (data_sample[i] - data_sample[i].mean()) / data_sample[i].std()
    
#     Leaving out of sample constant series
    if data_sample[i].isnull().sum().sum() != 0:
        print(i)
        indeces = pd.isnull(data_sample[i].transpose()).any(1).nonzero()[0]
        print(indeces)
        for ind in indeces:
            del data_sample[i][ind]

    data_sample[i].index = np.arange(len(data_sample[i]))
# Checking NANs or constant series
for data in data_sample:
    print(data.isnull().sum().sum())
    inds = pd.isnull(data.transpose()).any(1).nonzero()[0]

0
[1134]
2
[ 995 1163]
0
0
0
0
0
0


In [7]:
# Saving ddata

data_sample = {'Yearly': data_sample[0], 'Quarterly': data_sample[1],
          'Monthly': data_sample[2], 'Weekly': data_sample[3] , 'Daily': data_sample[4], 'Hourly': data_sample[5]}

for key in data_sample.keys():
    data_sample[key].to_csv(key+'_sample.csv')